<img style="float: right;" src="logo.png" width="125">

# Environment Finder
A tool for finding and analyzing atomic environments in crystal structures

In [1]:
import numpy as np
from ipywidgets import interactive, widgets, HTML
import warnings
import os
import markdown

warnings.simplefilter('ignore')

In [2]:
html_description = markdown.markdown("""
The purpose of this tool is to find atomic environments in crystal structures. These environments can then be used to define measures of similarity and collective variables for enhanced sampling simulations. The output from this tool can be used directly to create reference environments for the [EnvironmentSimilarity](https://www.plumed.org/doc-master/user-doc/html/_e_n_v_i_r_o_n_m_e_n_t_s_i_m_i_l_a_r_i_t_y.html) collective variable in [PLUMED](https://www.plumed.org/doc-master/user-doc/html/index.html). The tool can also be used to visualize chemical environments around an atom for general purposes.
""")
description=HTML(html_description)

pre_accordion = widgets.Accordion()
pre_accordion.children = [description]
pre_accordion.set_title(0, 'Description')
pre_accordion.selected_index = None
pre_accordion

Accordion(children=(HTML(value='<p>The purpose of this tool is to find atomic environments in crystal structur…

Proceed through the four tabs sequentially.

In [3]:
from environmentfinder import EnvironmentFinder
# Define instance of class EnvironmentFinder
MyEnvironmentFinder = EnvironmentFinder()

_ColormakerRegistry()

In [4]:
#############################
# WIDGET 1: Choose and upload
#############################

# Define upload widget
Widget1UploadConfiguration = widgets.FileUpload(
 accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
 multiple=False,  # True to accept multiple files upload else False
 wait=True
)

examples = ('Ice Ih', 'examples/IceIh.pdb'),('Urea', 'examples/urea2.pdb'), ('Ga II', 'examples/Ga_II.vasp')
widget_choose_filename = widgets.Dropdown(options=examples,description='Structure:')

Widget1Out = widgets.Output()
def chooseAndPlotConfigurationAltOutput(filename):
    Widget1Out.clear_output()
    with Widget1Out:
        MyEnvironmentFinder.chooseAndPlotConfiguration(filename)
        
Widget1ExampleConfiguration = interactive(chooseAndPlotConfigurationAltOutput, filename=widget_choose_filename)

# This function updates the dropdown list when a new file is uploaded
def updateWidget1ExampleConfiguration(*args):
    uploaded_filename = next(iter(Widget1UploadConfiguration.value))
    content = Widget1UploadConfiguration.value[uploaded_filename]['content']
    with open("Uploaded/" + uploaded_filename, 'wb') as f: f.write(content)
    mypath="Uploaded/"
    found_files=[]
    for f in os.listdir(mypath):
        if (f!="README.md"):
            found_files += [(f , mypath + f)]
    found_files = tuple(found_files)
    #found_files = tuple([(f , mypath + f) for f in os.listdir(mypath)])
    examples = ('Ice Ih', 'examples/IceIh.pdb'),('Urea', 'examples/urea2.pdb'), ('Ga II', 'examples/Ga_II.vasp')
    all_files = found_files+examples
    widget_choose_filename.options = all_files
    
Widget1UploadConfiguration.observe(updateWidget1ExampleConfiguration,names='value')

Widget1ExamplesAndUpload = widgets.HBox([Widget1ExampleConfiguration,Widget1UploadConfiguration])
Widget1Text = HTML(markdown.markdown("""
Choose a structure from the examples or upload your own in xyz, pdb, vasp, lammps-data, or [other formats](https://wiki.fysik.dtu.dk/ase/ase/io/io.html).
"""))
Widget1 = widgets.VBox([Widget1Text,Widget1ExamplesAndUpload,Widget1Out])

In [5]:
#############################
# WIDGET 2: Define
#############################

MyEnvironmentFinder.chooseConfiguration('examples/IceIh.pdb') # Have to load the first example
widget_atom_type_1 = widgets.Dropdown(options=MyEnvironmentFinder.atom_types, value=MyEnvironmentFinder.atom_types[1], description='Central atoms type:', style = {'description_width': 'initial'})
widget_atom_type_2 = widgets.Dropdown(options=MyEnvironmentFinder.atom_types, value=MyEnvironmentFinder.atom_types[0], description='Neighbor atoms type:', style = {'description_width': 'initial'})

def update_atom_types(*args):
    widget_atom_type_1.options = MyEnvironmentFinder.atom_types
    widget_atom_type_1.value = MyEnvironmentFinder.atom_types[0]
    widget_atom_type_2.options = MyEnvironmentFinder.atom_types
    widget_atom_type_2.value = MyEnvironmentFinder.atom_types[0]

widget_choose_filename.observe(update_atom_types, names='value')

#def toggleTypeAndIndex(value, fast):
def toggleTypeAndIndex(value):
    MyEnvironmentFinder.fastFlag=False
    if (value=='Type'):
        # Call function with widgets
        widget_define_type_pre = interactive(MyEnvironmentFinder.calculateEnvironmentsType, 
            atom_type_1 = widget_atom_type_1,
            atom_type_2 = widget_atom_type_2,
            cutoff =  widgets.FloatText(value=2,description='Cutoff (Å):',disabled=False),
            tolerance = widgets.FloatText(value=0.02,description='Tolerance (Å):',disabled=False, style = {'description_width': 'initial'}) 
        )
        widget_define_type_text = widgets.Label(value="Choose the atom type of the central and neighbor atoms, the cutoff, and the similarity tolerance.")
        widget_define_type = widgets.VBox([widget_define_type_text,widget_define_type_pre]) 
        display(widget_define_type)
    elif (value=='String'):
        # Call function with widgets
        widget_define_string_pre = interactive(MyEnvironmentFinder.calculateEnvironmentsString, 
                   listastring = widgets.Text(value="1,2,3", description='List central atoms:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   listbstring = widgets.Text(value="1,2", description='List neighbor atoms:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff (Å):',disabled=False, style = {'description_width': 'initial'}),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance (Å):',disabled=False, style = {'description_width': 'initial'}) 
        )
        widget_define_string_text = widgets.Label(value="Choose central and neighbor atoms using lists, then choose the cutoff and the similarity tolerance.")
        widget_define_string = widgets.VBox([widget_define_string_text,widget_define_string_pre])
        display(widget_define_string)
    elif (value=='Step'):
        # Call function with widgets
        widget_define_step_pre = interactive(MyEnvironmentFinder.calculateEnvironmentsMinMaxStride,
                   mina = widgets.Text(value="1", description='Central atoms min:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   maxa = widgets.Text(value="2", description='Central atoms max:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   stridea = widgets.Text(value="1", description='Central atoms step:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   minb = widgets.Text(value="1", description='Neighbor atoms min:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   maxb = widgets.Text(value="2", description='Neighbor atoms max:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   strideb = widgets.Text(value="1", description='Neighbor atoms step:',placeholder='Type something',disabled=False, style = {'description_width': 'initial'}),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff (Å):',disabled=False, style = {'description_width': 'initial'}),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance (Å):',disabled=False, style = {'description_width': 'initial'}) 
        )
        widget_define_step_text = widgets.Label(value="Choose central and neighbor atoms using minimum and maximum indeces and a step. Then choose the cutoff and the similarity tolerance.")
        widget_define_step = widgets.VBox([widget_define_step_text,widget_define_step_pre])
        display(widget_define_step)
    else:
        print("Error")

Widget2Pre = interactive(toggleTypeAndIndex, value=widgets.ToggleButtons(options=['Type','String','Step'], description='Choose based on:', style = {'description_width': 'initial'}, disabled=False))

#                                       fast=widgets.Checkbox(value=False, description="Fast algorithm", disabled=False, style = {'description_width': 'initial'}))

Widget2Text = widgets.Label(value="Define the environments. The tool will then find the unique environments in the configuration using your definition.")
Widget2 = widgets.VBox([Widget2Text,Widget2Pre])

/home/local/PRINCETON/ppiaggi/miniconda3/lib/python3.8/site-packages/traitlets/traitlets.py:714: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


In [ ]:
#############################
# WIDGET 3: Analyze
#############################

Widget3Out = widgets.Output()

def update3(*args):
    toggleAllVsUnique('Unique')
        
def toggleAllVsUnique(value):
    Widget3Out.clear_output()
    if (value=='Unique' and MyEnvironmentFinder.uniqueEnvs.shape[0]>0  and MyEnvironmentFinder.uniqueFlag):
        plotMyEnv = interactive(MyEnvironmentFinder.chooseEnvPlotUnique, number=widgets.IntSlider(description='Environment:',min=1,max=MyEnvironmentFinder.uniqueEnvs.shape[0],step=1,value=0)) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        with Widget3Out:
            display(plotMyEnv)
    elif (value=='All' and MyEnvironmentFinder.allEnvs.shape[0]>0):
        plotMyEnv = interactive(MyEnvironmentFinder.chooseEnvPlotAll, number=widgets.IntSlider(description='Environment:',min=1,max=MyEnvironmentFinder.allEnvs.shape[0],step=1,value=0, style = {'description_width': 'initial'})) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        with Widget3Out:
            display(plotMyEnv)
    elif (value!='All' and value!='Unique'):
        with Widget3Out:
            print("Error: keyword " + str(value) + " not recognized!")
    elif (not(MyEnvironmentFinder.uniqueFlag)):
        with Widget3Out:
            print("Error: unique environments not requested!")
    elif (MyEnvironmentFinder.allEnvs.shape[0]==0 or MyEnvironmentFinder.uniqueEnvs.shape[0]==0):
        with Widget3Out:
            print("Error: empty environments!")
    else:
        with Widget3Out:
            print("Error")
        
Widget3AnalyzeEnvironmentsToggle = interactive(toggleAllVsUnique, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

Widget3Text = widgets.Label(value='Visualize the calculated environments. Toggle between the unique environments or all, and slide to choose environment.')

Widget3AnalyzeEnvironments = widgets.VBox([Widget3Text,Widget3AnalyzeEnvironmentsToggle,Widget3Out])

In [ ]:
#############################
# WIDGET 4: Output
#############################

Widget4Out = widgets.Output()

def update4(*args):
    toggleAllVsUniqueForOutput('Unique')
        
def toggleAllVsUniqueForOutput(value):
    Widget4Out.clear_output()
    if (value=='Unique' and MyEnvironmentFinder.uniqueEnvs.shape[0]>0 and MyEnvironmentFinder.uniqueFlag):
        with Widget4Out:
            MyEnvironmentFinder.printEnvironments(MyEnvironmentFinder.uniqueEnvs)
            MyEnvironmentFinder.printEnvironmentsToZipFile(MyEnvironmentFinder.uniqueEnvs)
    elif (value=='All' and MyEnvironmentFinder.allEnvs.shape[0]>0):
        with Widget4Out:
            MyEnvironmentFinder.printEnvironments(MyEnvironmentFinder.allEnvs)
            MyEnvironmentFinder.printEnvironmentsToZipFile(MyEnvironmentFinder.allEnvs)
    elif (value!='All' and value!='Unique'):
        with Widget4Out:
            print("Error: keyword " + str(value) + " not recognized!")
    elif (not(MyEnvironmentFinder.uniqueFlag)):
        with Widget4Out:
            print("Error: unique environments not requested!")
    elif (MyEnvironmentFinder.allEnvs.shape[0]==0 or MyEnvironmentFinder.uniqueEnvs.shape[0]==0):
        with Widget4Out:
            print("Error: empty environments!")
    else:
        with Widget4Out:
            print("Error")
        
Widget4OutputEnvironmentsToggle = interactive(toggleAllVsUniqueForOutput, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

Widget4Text = widgets.Label(value='Print the environments in Protein Data Bank (PDB) format. Toggle between the unique environments or all.')

Widget4TextPost = HTML(markdown.markdown("""
These environments can be used in an enhanced sampling simulation using the [EnvironmentSimilarity](https://www.plumed.org/doc-master/user-doc/html/_e_n_v_i_r_o_n_m_e_n_t_s_i_m_i_l_a_r_i_t_y.html) collective variable in [PLUMED](https://www.plumed.org/doc-master/user-doc/html/index.html).
"""))

#Widget4Button = widgets.Button(
#    description='Download',
#    disabled=False,
#    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#    tooltip='Download',
#    icon='download' # (FontAwesome names without the `fa-` prefix)
#)
#
#def zip_and_download(dummy):
#    MyEnvironmentFinder.printEnvironmentsToFile(MyEnvironmentFinder.uniqueEnvs)
#
#Widget4Button.on_click(zip_and_download)

Widget4OutputEnvironments = widgets.VBox([Widget4Text,Widget4OutputEnvironmentsToggle,Widget4Out,Widget4TextPost])

In [ ]:
#############################
# Combine widgets into tabs
#############################

children = [Widget1,Widget2,Widget3AnalyzeEnvironments,Widget4OutputEnvironments]
tab = widgets.Tab() #layout=widgets.Layout(width='800px', height='800px'))
tab.children = children
tab.set_title(0, 'Choose configuration')
tab.set_title(1, 'Define environments')
tab.set_title(2, 'Analyze environments')
tab.set_title(3, 'Output environments')


tab.observe(update3, names='selected_index')
tab.observe(update4, names='selected_index')

In [ ]:
display(tab)

In [ ]:
html_acknowledgments = markdown.markdown("""
* The app uses several python libraries, for instance [ASE](https://wiki.fysik.dtu.dk/ase/), [NGLVIEW](https://github.com/arose/nglview), and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html).
* I am grateful to Giovanni Pizzi and Dou Du for suggesting to deploy the tool using [Binder](https://mybinder.org/)+[appmode](https://github.com/oschuett/appmode).
* This tool was mainly developed with support of the Swiss National Science Foundation (SNSF) through an Early Postdoc.Mobility fellowship.
* I also acknowledge funding from the NCCR MARVEL funded by the SNSF and from the CSI Computational Science Center funded by the Department of Energy of the USA.
""")
acknowledgments=widgets.VBox([
    HTML(html_acknowledgments)
])

html_howtocite = markdown.markdown("""
If you are using this tool to find environments for enhanced sampling simulations please read and cite:

* [Pablo Piaggi and Michele Parrinello, *Calculation of phase diagrams in the multithermal-multibaric ensemble*, J. Chem. Phys. 150, 244119 (2019)](https://aip.scitation.org/doi/full/10.1063/1.5102104)
""")
howtocite=widgets.VBox([
    HTML(html_howtocite)
])

post_children=[acknowledgments,howtocite]
post_accordion = widgets.Accordion(layout=widgets.Layout())
#print(post_accordion.layout.keys)
post_accordion.children = post_children
post_accordion.set_title(0, 'Acknowledgments')
post_accordion.set_title(1, 'How to cite')
post_accordion.selected_index = None
post_accordion

See the project on [GitHub](https://github.com/PabloPiaggi/EnvironmentFinder)

Environment Finder, Copyright © 2020, Pablo Piaggi